In [61]:
#This script if for investigating how deep of a circuit you can run with an IBM device using the corresponding backend noise model
import sys
import json
import numpy as np
from scipy.optimize import minimize
import time

# Packages for quantum stuff
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    QiskitRuntimeService,
    SamplerV2 as Sampler,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import (
    FakeBrisbane,
    FakeSherbrooke,
    FakeTorino,
)  # For simulation with realistic noise

In [62]:
# //////////    Variables    //////////
desiredProblemType = 'Knapsack' #options: 'Knapsack'
targetQubitRange = range(2,10+1) #range of qubits to test
reps_p = 1

repsPerProblemScale = 10 
# This is the number of times that the optimsation of parameters is repeated for each qubit number.
# This avoids issues with parameter initalisation. Should be 100 for final test

backend_simulator = AerSimulator()
# backend_simulator = AerSimulator.from_backend(FakeSherbrooke())
# backend_simulator = AerSimulator.from_backend(FakeBrisbane())
# backend_simulator = AerSimulator.from_backend(FakeTorino())

In [63]:
# //////////    Functions    //////////
def load_qubo_and_build_hamiltonian(file_path):
    """
    Loads QUBO terms, weights, and constant from a JSON file.
    Determines the number of qubits from the terms and constructs
    the Hamiltonian as a Qiskit SparsePauliOp.
    """
    with open(file_path, "r") as f:
        all_qubo_data = json.load(f)

    if isinstance(all_qubo_data, list):
        # If it's a list, take the first element
        qubo_data = all_qubo_data[0]
    else:
        # If it's already a dictionary, just use it directly
        qubo_data = all_qubo_data


    terms = qubo_data["terms"]
    weights = qubo_data["weights"]
    constant = qubo_data.get("constant", 0.0)
    problemType = qubo_data.get("problem_type")

    pauli_list = []
    num_qubits = 0

    if terms:
        # Flatten the list of lists and filter out empty sublists or non-integer elements
        all_indices = []
        for term_group in terms:
            if isinstance(term_group, list): # Ensure it's a list
                for idx in term_group:
                    if isinstance(idx, int): # Ensure index is an integer
                        all_indices.append(idx)

        if all_indices: # If there are any valid integer indices
            num_qubits = max(all_indices) + 1
        else: # No indices and no weights (only constant)
            num_qubits = 0
    else: # No terms at all
        num_qubits = 0
        if weights: # Weights present but no terms - problematic
            print("Warning: Weights are present, but 'terms' list is empty or missing. Cannot form Pauli operators.")

    for term_indices, weight in zip(terms, weights):
        if not term_indices or not all(isinstance(idx, int) for idx in term_indices):
            # Skip if term_indices is empty or contains non-integers
            continue

        paulis_arr = ["I"] * num_qubits
        if len(term_indices) == 1: # Linear term
            paulis_arr[term_indices[0]] = "Z"
        elif len(term_indices) == 2: # Quadratic term
            paulis_arr[term_indices[0]] = "Z"
            paulis_arr[term_indices[1]] = "Z"
        else:
            # This case should ideally not be hit if terms are only single or pairs.
            print(f"Warning: Skipping term {term_indices} with unsupported number of variables for Pauli Z construction.")
            continue
        pauli_list.append(("".join(paulis_arr)[::-1], weight))

    if not pauli_list and num_qubits > 0: # No valid Pauli terms were created, but num_qubits > 0
        cost_hamiltonian = SparsePauliOp(["I"] * num_qubits, [0]) # Zero operator on n_qubits
    elif not pauli_list and num_qubits == 0:
        cost_hamiltonian = SparsePauliOp("I", [0]) # Placeholder for 1 qubit if everything is empty
    else:
        cost_hamiltonian = SparsePauliOp.from_list(pauli_list)

    return cost_hamiltonian, constant, num_qubits, problemType

def cost_func_estimator(params, ansatz, estimator, cost_hamiltonian_logical, constant_offset, backend_total_qubits=127): # removed default for backend_total_qubits
    global numOptimisations
    prepared_observable = cost_hamiltonian_logical.apply_layout(ansatz.layout)
    pub = (ansatz, prepared_observable, [params])
    
    job = estimator.run(pubs=[pub])
    results = job.result()[0]
    cost = results.data.evs[0]

    cost_float = float(np.real(cost)) + constant_offset
    
    return cost_float


In [64]:
# Training the QAOA for each problem scale and getting the best parameters for each scale achieved after 'repsPerProblemScale' runs
bestParameters = {}
for qubitNum in targetQubitRange:
    #file bizniz
    filename = f"QUBO_batches/batch_QUBO_data_{desiredProblemType}_{qubitNum}q_.json"
    print(f"Training QAOA for first QUBO in {filename}...")
    cost_hamiltonian, constant_offset, num_qubits, problem_type = load_qubo_and_build_hamiltonian(filename)
    #print(f"Number of qubits (inferred from terms): {num_qubits}")

    #circuit compilation bizniz
    estimator = Estimator(mode=backend_simulator)
    pm = generate_preset_pass_manager(optimization_level=3, backend=backend_simulator)
    circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=reps_p)
    circuit.measure_all() 
    pm = generate_preset_pass_manager(optimization_level=3, backend=backend_simulator)
    candidate_circuit = pm.run(circuit)
    estimator = Estimator(mode=backend_simulator)
    transpiledCircuitDepth = candidate_circuit.depth()
    print(f"Transpiled circuit depth: {transpiledCircuitDepth}")

    #keeping track of parameters and associated cost function values produced so far for this problem scale
    allResults = []

    for repitition in range(1, repsPerProblemScale+1):

        #creating random inital parameters
        initial_betas = (np.random.rand(reps_p) * np.pi).tolist()
        initial_gammas = (np.random.rand(reps_p) * np.pi).tolist()
        initial_params = initial_betas + initial_gammas

        #training QAOA parameters
        #numOptimisations = 0
        print(f"\rRunning repitition {repitition}/{repsPerProblemScale} for {qubitNum} qubit {desiredProblemType} problem...", end="")
        result = minimize(
            cost_func_estimator,
            initial_params,
            args=(candidate_circuit, estimator, cost_hamiltonian, constant_offset),
            method="COBYLA",
            tol=1e-3,
            options={"maxiter": 1000}, # Adjust as needed
        )
        allResults.append([result.fun, result.x])
     
    qubitWiseBestResult = min(allResults, key=lambda item: item[0])
    qubitWiseBestParameters = qubitWiseBestResult[1]
    bestParameters[qubitNum] = {
    "transpiledDepth": transpiledCircuitDepth,
    "bestParamsForScale": qubitWiseBestParameters
}

print(f"\nBest betas and gammas for each problem scale: {bestParameters}")

Training QAOA for first QUBO in QUBO_batches/batch_QUBO_data_Knapsack_2q_.json...
Transpiled circuit depth: 5
Running repitition 10/10 for 2 qubit Knapsack problem...Training QAOA for first QUBO in QUBO_batches/batch_QUBO_data_Knapsack_3q_.json...
Transpiled circuit depth: 7
Running repitition 10/10 for 3 qubit Knapsack problem...Training QAOA for first QUBO in QUBO_batches/batch_QUBO_data_Knapsack_4q_.json...
Transpiled circuit depth: 9
Running repitition 10/10 for 4 qubit Knapsack problem...Training QAOA for first QUBO in QUBO_batches/batch_QUBO_data_Knapsack_5q_.json...
Transpiled circuit depth: 11
Running repitition 10/10 for 5 qubit Knapsack problem...Training QAOA for first QUBO in QUBO_batches/batch_QUBO_data_Knapsack_6q_.json...
Transpiled circuit depth: 12
Running repitition 10/10 for 6 qubit Knapsack problem...Training QAOA for first QUBO in QUBO_batches/batch_QUBO_data_Knapsack_7q_.json...
Transpiled circuit depth: 14
Running repitition 10/10 for 7 qubit Knapsack problem...T

In [ ]:
for qubitNum in targetQubitRange:
    optimized_circuit = candidate_circuit.assign_parameters(bestParameters[qubitNum]["bestParamsForScale"])
    sampler = Sampler(mode=backend_simulator)
    sampler.options.default_shots = 1000
    

[2.73864277 2.65832855]
[0.72323355 3.86802582]
[2.84988687 0.23831937]
[3.91946627 4.73569144]
[1.35665648 3.18096572]
[3.86218276 1.67210793]
[5.53287158 1.96871456]
[1.28193394 2.35841326]
[0.67047823 2.22367339]
